In [1]:
import os
import csv
import pandas as pd
from datetime import datetime
import cassandra as cas

In [2]:
ls

 Volume in drive D is Local Disk 
 Volume Serial Number is 407D-3EE5

 Directory of D:\data-science\towards-data-science\3__databases\in3__cassandra

19-12-2022  22:11    <DIR>          .
19-12-2022  22:11    <DIR>          ..
18-12-2022  19:29    <DIR>          .ipynb_checkpoints
18-12-2022  18:43           494,508 in1__intro to cassandra.ipynb
19-12-2022  22:11            71,341 in2__task (dump bulk data to DataStax Astra).ipynb
18-12-2022  14:52            12,265 secure-connect-cassandra-tutorial.zip
19-12-2022  18:44           362,669 wafer_23012020_041211.csv
               4 File(s)        940,783 bytes
               3 Dir(s)  31,497,228,288 bytes free


In [3]:
## Establish connection to DataStax Astra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    'secure_connect_bundle': 'secure-connect-cassandra-tutorial.zip'
}
auth_provider = PlainTextAuthProvider(
    'fcvBmNLRmCdfXYuTkeZZedtZ', 
    'U.jS0oFiT739KE9wUPHNrrbCtH9axPS15dsTDmQ6Q62Mgw7TXIfbmdNE2NhKtppxo3ZfF8pMX3Sod5XaQiuC,G9Ag1QFofJru9_3G8y-iOvecKhoQvFgAJLJFxL6u.+2')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('datasets')
row = session.execute("select release_version from system.local").one()
if row:
    print("release_version: ", row)
else:
    print("An error occurred.")

release_version:  Row(release_version='4.0.0.6816')


In [4]:
## Let's see what we can do about the csv file that's to be dumped

with open('wafer_23012020_041211.csv', 'r') as f:
    print(f.readline())

Unnamed: 0,Sensor-1,Sensor-2,Sensor-3,Sensor-4,Sensor-5,Sensor-6,Sensor-7,Sensor-8,Sensor-9,Sensor-10,Sensor-11,Sensor-12,Sensor-13,Sensor-14,Sensor-15,Sensor-16,Sensor-17,Sensor-18,Sensor-19,Sensor-20,Sensor-21,Sensor-22,Sensor-23,Sensor-24,Sensor-25,Sensor-26,Sensor-27,Sensor-28,Sensor-29,Sensor-30,Sensor-31,Sensor-32,Sensor-33,Sensor-34,Sensor-35,Sensor-36,Sensor-37,Sensor-38,Sensor-39,Sensor-40,Sensor-41,Sensor-42,Sensor-43,Sensor-44,Sensor-45,Sensor-46,Sensor-47,Sensor-48,Sensor-49,Sensor-50,Sensor-51,Sensor-52,Sensor-53,Sensor-54,Sensor-55,Sensor-56,Sensor-57,Sensor-58,Sensor-59,Sensor-60,Sensor-61,Sensor-62,Sensor-63,Sensor-64,Sensor-65,Sensor-66,Sensor-67,Sensor-68,Sensor-69,Sensor-70,Sensor-71,Sensor-72,Sensor-73,Sensor-74,Sensor-75,Sensor-76,Sensor-77,Sensor-78,Sensor-79,Sensor-80,Sensor-81,Sensor-82,Sensor-83,Sensor-84,Sensor-85,Sensor-86,Sensor-87,Sensor-88,Sensor-89,Sensor-90,Sensor-91,Sensor-92,Sensor-93,Sensor-94,Sensor-95,Sensor-96,Sensor-97,Sensor-98,Sensor-99,Sensor-1

In [8]:
with open('wafer_23012020_041211.csv', 'r') as f:
    data = csv.reader(f, delimiter=',')
    print(type(data))  # kinda generator
    count = 0
    for row in data:
        print(row)
        count += 1
        if count > 5:  # Let's just stick for now to first few records
            break

<class '_csv.reader'>
['', 'Sensor-1', 'Sensor-2', 'Sensor-3', 'Sensor-4', 'Sensor-5', 'Sensor-6', 'Sensor-7', 'Sensor-8', 'Sensor-9', 'Sensor-10', 'Sensor-11', 'Sensor-12', 'Sensor-13', 'Sensor-14', 'Sensor-15', 'Sensor-16', 'Sensor-17', 'Sensor-18', 'Sensor-19', 'Sensor-20', 'Sensor-21', 'Sensor-22', 'Sensor-23', 'Sensor-24', 'Sensor-25', 'Sensor-26', 'Sensor-27', 'Sensor-28', 'Sensor-29', 'Sensor-30', 'Sensor-31', 'Sensor-32', 'Sensor-33', 'Sensor-34', 'Sensor-35', 'Sensor-36', 'Sensor-37', 'Sensor-38', 'Sensor-39', 'Sensor-40', 'Sensor-41', 'Sensor-42', 'Sensor-43', 'Sensor-44', 'Sensor-45', 'Sensor-46', 'Sensor-47', 'Sensor-48', 'Sensor-49', 'Sensor-50', 'Sensor-51', 'Sensor-52', 'Sensor-53', 'Sensor-54', 'Sensor-55', 'Sensor-56', 'Sensor-57', 'Sensor-58', 'Sensor-59', 'Sensor-60', 'Sensor-61', 'Sensor-62', 'Sensor-63', 'Sensor-64', 'Sensor-65', 'Sensor-66', 'Sensor-67', 'Sensor-68', 'Sensor-69', 'Sensor-70', 'Sensor-71', 'Sensor-72', 'Sensor-73', 'Sensor-74', 'Sensor-75', 'Sensor

In [5]:
## Read the dataset as pandas dataframe
# Convert na vals to NULL and save the dataframe.

df = pd.read_csv("wafer_23012020_041211.csv")
df.fillna('NULL', inplace=True)
df.to_csv("wafer_23012020_041211.csv", index=None)

df.head()

,Unnamed: 0,Sensor-1,Sensor-2,Sensor-3,Sensor-4,Sensor-5,Sensor-6,Sensor-7,Sensor-8,Sensor-9,...,Sensor-582,Sensor-583,Sensor-584,Sensor-585,Sensor-586,Sensor-587,Sensor-588,Sensor-589,Sensor-590,Good/Bad
0,Wafer-801,2968.33,2476.58,2216.7333,1748.0885,1.1127,100.0,97.5822,0.1242,1.5300,...,NULL,0.5004,0.0120,0.0033,2.4069,0.0545,0.0184,0.0055,33.7876,-1
1,Wafer-802,2961.04,2506.43,2170.0666,1364.5157,1.5447,100.0,96.77,0.123,1.3953,...,NULL,0.4994,0.0115,0.0031,2.3020,0.0545,0.0184,0.0055,33.7876,1
2,Wafer-803,3072.03,2500.68,2205.7445,1363.1048,1.0518,100.0,101.8644,0.122,1.3896,...,NULL,0.4987,0.0118,0.0036,2.3719,0.0545,0.0184,0.0055,33.7876,-1
3,Wafer-804,3021.83,2419.83,2205.7445,1363.1048,1.0518,100.0,101.8644,0.122,1.4108,...,NULL,0.4934,0.0123,0.0040,2.4923,0.0545,0.0184,0.0055,33.7876,-1
4,Wafer-805,3006.95,2435.34,2189.8111,1084.6502,1.1993,100.0,104.8856,0.1234,1.5094,...,NULL,0.4987,0.0145,0.0041,2.8991,0.0545,0.0184,0.0055,33.7876,-1


In [6]:
## dataset info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 592 entries, Unnamed: 0 to Good/Bad
dtypes: float64(327), int64(97), object(168)
memory usage: 462.6+ KB


In [9]:
cols = df.columns
cols

table_cols = ["Wafer text primary key, "]
for i in range(len(cols)):
    if i == 0:
        continue
    elif i != len(cols) - 1:
        col_name = f'"{cols[i]}" float, '
        table_cols[0] += col_name
    else:
        col_name = f'"{cols[i]}" int'
        table_cols[0] += col_name
    
table_cols[0]

'Wafer text primary key, "Sensor-1" float, "Sensor-2" float, "Sensor-3" float, "Sensor-4" float, "Sensor-5" float, "Sensor-6" float, "Sensor-7" float, "Sensor-8" float, "Sensor-9" float, "Sensor-10" float, "Sensor-11" float, "Sensor-12" float, "Sensor-13" float, "Sensor-14" float, "Sensor-15" float, "Sensor-16" float, "Sensor-17" float, "Sensor-18" float, "Sensor-19" float, "Sensor-20" float, "Sensor-21" float, "Sensor-22" float, "Sensor-23" float, "Sensor-24" float, "Sensor-25" float, "Sensor-26" float, "Sensor-27" float, "Sensor-28" float, "Sensor-29" float, "Sensor-30" float, "Sensor-31" float, "Sensor-32" float, "Sensor-33" float, "Sensor-34" float, "Sensor-35" float, "Sensor-36" float, "Sensor-37" float, "Sensor-38" float, "Sensor-39" float, "Sensor-40" float, "Sensor-41" float, "Sensor-42" float, "Sensor-43" float, "Sensor-44" float, "Sensor-45" float, "Sensor-46" float, "Sensor-47" float, "Sensor-48" float, "Sensor-49" float, "Sensor-50" float, "Sensor-51" float, "Sensor-52" flo

In [10]:
## Create a table `wafer` in relevant keyspace

session.execute(
    f"""
    create table if not exists wafer
    ({table_cols[0]}
    );
    """).one()

InvalidRequest: Error from server: code=2200 [Invalid query] message="Tables cannot have more than 75 columns, but 592 provided for table wafer"

In [10]:
with open('annual-enterprise-survey-2020-financial-year-provisional-csv.csv', 'r') as f:
    data = csv.reader(f, delimiter=',')
    next(data)
    
    count = 0
    for row in data:
        insertable_row = [f"{count}, "]
        # prepare row in an insertable form
        for ele_idx in range(len(row)):
            if ele_idx == 0:
                insertable_row[0] += f'{int(row[ele_idx])}, '
            else:
                insertable_row[0] += f'"{row[ele_idx]}", '            
            
        insertable_row = insertable_row[0][:-2]
        print(insertable_row)
        
        # dump insertable row in cassandra table
        session.execute(
        f"""
        insert into survey_data({table_cols[0]})
        values ({insertable_row})
        """)
        count += 1
        if count > 10:  # Let's just stick for now to the first 10 records
            break

0, 2020, "Level 1", "99999", "All industries", "Dollars (millions)", "H01", "Total income", "Financial performance", "733,258", "ANZSIC06 divisions A-S (excluding classes K6330, L6711, O7552, O760, O771, O772, S9540, S9601, S9602, and S9603)"


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 2:35 mismatched input 'int' expecting ')' (        insert into survey_data(id [int]...)">

=> yeah, right, seriously?!